[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_garch_estimation/EMQA_garch_estimation.ipynb)

# EMQA_garch_estimation
GARCH parameter estimation and interpretation -- visualizing omega, alpha, beta and persistence.
**Output:** `garch_estimation.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent and fit GARCH
brent = fetch('BZ=F', start='2018-01-01')
log_ret = (np.log(brent / brent.shift(1)).dropna()) * 100

try:
    from arch import arch_model
    am = arch_model(log_ret, vol='Garch', p=1, q=1, dist='normal', mean='Constant')
    res = am.fit(disp='off')
    omega = res.params['omega']
    alpha = res.params['alpha[1]']
    beta_ = res.params['beta[1]']
    use_arch = True
except ImportError:
    omega, alpha, beta_ = 0.01, 0.08, 0.90
    use_arch = False

persistence = alpha + beta_
print(f"omega  = {omega:.6f}")
print(f"alpha  = {alpha:.4f}")
print(f"beta   = {beta_:.4f}")
print(f"alpha+beta (persistence) = {persistence:.4f}")

In [ ]:
# 3-panel: Parameter bar chart, Persistence bar, News impact curve
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Panel 1 - Parameter bar chart
params = ['$\\omega$', '$\\alpha$', '$\\beta$']
values = [omega, alpha, beta_]
bar_colors = [COLORS['blue'], COLORS['red'], COLORS['green']]
bars = axes[0].bar(params, values, color=bar_colors, width=0.5, edgecolor='white')
for bar, val in zip(bars, values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                 f'{val:.4f}', ha='center', va='bottom', fontsize=10)
axes[0].set_ylabel('Parameter Value')
axes[0].set_title('GARCH(1,1) Parameters')

# Panel 2 - Persistence gauge (horizontal bar)
axes[1].barh(['Persistence\n($\\alpha+\\beta$)'], [persistence], color=COLORS['orange'],
             height=0.4, edgecolor='white')
axes[1].barh(['Persistence\n($\\alpha+\\beta$)'], [1 - persistence], left=[persistence],
             color=COLORS['gray'], alpha=0.2, height=0.4, edgecolor='white')
axes[1].set_xlim(0, 1.05)
axes[1].set_xlabel('Value')
axes[1].set_title(f'Volatility Persistence = {persistence:.4f}')
axes[1].axvline(1.0, color=COLORS['red'], linestyle='--', linewidth=1, alpha=0.5)

# Panel 3 - News impact curve from fitted parameters
h_unc = omega / (1 - alpha - beta_) if persistence < 1 else omega / 0.01
eps = np.linspace(-4, 4, 500)
nic = omega + alpha * eps**2 + beta_ * h_unc
axes[2].plot(eps, nic, color=COLORS['purple'], linewidth=2)
axes[2].axvline(0, color=COLORS['gray'], linestyle='--', linewidth=0.8, alpha=0.5)
axes[2].set_xlabel('Standardized Shock ($\\epsilon_{t-1}$)')
axes[2].set_ylabel('Conditional Variance ($h_t$)')
axes[2].set_title('Fitted News Impact Curve')

plt.tight_layout()
save_fig(fig, 'garch_estimation.pdf')
plt.show()